# Generating and evaluating ML model for segmenting singledoubletriple vertical lines

ML model did not end up being used in any paper

## Initialization

In [ ]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
sys.path.append(os.path.dirname(module_path))
from py.tools.config import cfg
import py.tools.logs as logs
import py.vid.v_tools as vt
import py.metrics.m_SDT as me
from py.im.imshow import imshow
import py.file.file_handling as fh
import pandas as pd
LOGGERDEFINED = logs.openLog('still_measure_vert_MLtrain.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

# export crops

In [ ]:
# export cropped images and table of their crop locations
folder = os.path.join(cfg.path.server, 'SDT\SO_S20-0.5')
exp = me.cropExporter(folder, me.fileVertSDT, mustMatch=['Vert'], overwrite=True)
exp.run()

# move crops to ML folder

In [ ]:
# find folders which are missing ML results and copy the files to the transfer folder
cropfolder = os.path.join(cfg.path.server, 'singleDoubleTripleML', 'newCrops')
topfolder = os.path.join(cfg.path.server, 'SDT')
missingFolders = me.findMissingML(topfolder, cropfolder, mustMatch=['Vert'])

In [5]:
# split newcrops into sub folders
folder = os.path.join(cfg.path.server, 'singleDoubleTripleML', 'newCrops')
me.splitIntoSubFolders(folder)

# move ML results back to folders

In [ ]:
# move ML segmented images from the output folder to the appropriate subfolders
folders = [os.path.join(cfg.path.server
                       , r'singleDoubleTripleML\results'
                       , f'lf-unet-vert-SO-{s}-segment-outputDir'
                       , 'png') for s in [10]]
rm = me.resultMover(folders,os.path.join(cfg.path.server, 'SDT'))

In [ ]:
rm.moveMLResult(rm.error[2])

# generate training and evaluation data

In [ ]:
tg = me.trainingGenerator(os.path.join(cfg.path.server, 'SDT')
                          , excludeFolders=[os.path.join(cfg.path.fig, 'SDT\trainingVert\segmented'),
                                          os.path.join(cfg.path.fig, 'SDT\trainingVert\evaluate\segmented')]
                          , mustMatch='disturbVert')

In [ ]:
file = tg.randomFile()

In [ ]:
vs = me.fileVertSDT(file, diag=1, overrideSegment=False)

In [ ]:
# add to training as is
vs.addToTraining(trainFolder=r'SDT\trainingVert\evaluate')

In [ ]:
# add to training, but open paint
vs.addToTraining(trainFolder=r'SDT\trainingVert\evaluate', openPaint=True)

In [ ]:
# add thresh to training and open paint
vs.addToTraining(s='thresh', trainFolder=r'SDT\trainingVert\evaluate')

In [ ]:
me.convertFilesToBW(os.path.join(cfg.path.fig, r'SDT\trainingVert\segmented'), diag=True)

# compare to manual segmentation

#### combined model

In [ ]:
scc = me.segmentCompare(os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\segmented')
                       , os.path.join(cfg.path.server, 'SDT')
                       , os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\orig')
                        , me.fileVertSDT)
scc.compare()

In [ ]:
scc.showWorstSegmentation()

In [ ]:
scc.compareFile(109, diag=4)

#### ML model

In [ ]:
scm = me.segmentCompare(os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\segmented')
                       , os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\png')
                       , os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\orig')
                        , 'ML')

scm.compare()

In [ ]:
scm.showWorstSegmentation()

In [ ]:
scm.compareFile(86, diag=2)

#### unsupervised model

In [ ]:
scu = me.segmentCompare(os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\segmented')
                       , os.path.join(cfg.path.server, 'SDT')
                       , os.path.join(cfg.path.server, r'singleDoubleTripleML\evaluateVert\orig')
                        , 'U')

scu.compare()

In [ ]:
scu.showWorstSegmentation()

In [ ]:
scu.compareFile(86, diag=2)